# 作业2 股票数据清洗

同学们好，本次作业的主要内容为使用pandas进行数据预处理。希望这两天你们已经从Python的课程内容中回过神来了。
没有数据的分析是无源之水，能够熟练处理数据是成为合格的数据分析师的基本要求，希望大家在今后多多实战，成为数据大师。

本次作业将使用公开标普500的股票数据。
https://www.kaggle.com/dgawlik/nyse#prices-split-adjusted.csv

作业的形式以问答为主，因为不涉及过长的代码，核心是数据的操作，所以这里没有太多的代码模板可供大家参考。
希望大家能从搜索引擎（google/bing）问答社区（stackoverflow）或者[官方文档](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html)或者我提供的jupyter notebooks 中找到回答问题需要的方法。
请时刻牢记数据的增删改查四种原子操作，思考我们的问题可以被如何分解。

那么首先，我们要import pandas

In [1]:
import pandas as pd

# 1. 从fundamentals.csv开始！

fundamentals.csv 是这些股票的年报数据

请用数据回答以下问题：

1. S&P500股票在2015年`net income`的均值是多少？最大值比最小值多多少？（每问10分，共计20分）
2. S&P500股票在2016年的固定资产（fixed assets）占总资产(total assets)比例的均值是多少？固定资产占总资产比例最小的股票是的代码（ticker symbol）是什么？（每问10分，共计20分）


In [2]:
fun_df = pd.read_csv('nyse/fundamentals.csv')
fun_df.rename(columns = {'For Year': 'For_Year'}, inplace = True)

In [3]:
_2015_data = fun_df.query('For_Year == 2015') # 筛选出2015年的数据
ni_mean = _2015_data['Net Income'].mean() # 2015年净收入（net income）的均值
print('2015年净收入的均值：{}'.format(ni_mean))
diff = _2015_data['Net Income'].max() - _2015_data['Net Income'].min() # 最大值比最小值多的
print('最大值比最小值多：{}'.format(diff))

2015年净收入的均值：1575185571.764706
最大值比最小值多：76922000000.0


In [4]:
_2016_data = fun_df.query('For_Year == 2016') # 筛选出2016年的数据
# 2016年的固定资产（fixed assets）占总资产(total assets)比例的均值
ft_proportion = (_2016_data['Fixed Assets'] / _2016_data['Total Assets']).mean()
print('固定资产占总资产比例的均值：{}'.format(ft_proportion))

_2016_data['ft_proportion'] = _2016_data['Fixed Assets'] / _2016_data['Total Assets'] # 在2016年的数据中新增一列'ft_proportion'
_2016_data.sort_values(by = ['ft_proportion'], inplace = True) # 按'ft_proportion'排序
print('固定资产占总资产比例最小的股票的代码：{}'.format(_2016_data.iloc[0]['Ticker Symbol']))

固定资产占总资产比例的均值：0.18557794433374705
固定资产占总资产比例最小的股票的代码：DHI


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


# 2. 加入securities.csv~

securities.csv包含了这些股票的基本信息

1. 请列举出各个sector中的加入时间最早的股票名称（10分）
2. 请列举出每一个州中加入时间最晚的股票名称（10分）


In [5]:
sec_df = pd.read_csv('nyse/securities.csv')
sec_df.dropna(axis = 0, how = 'any', inplace = True) # 处理缺失数据

In [6]:
sec_df.sort_values(by = ['Date first added'], inplace=True) # 按'Data first added'对数据排序
for key, value in sec_df.groupby('GICS Sector'): # 按'GICS Sector'分组并迭代
    print('{} 加入时间最早的股票名称：{}'.format(key, value['Ticker symbol'].iloc[0]))

Consumer Discretionary 加入时间最早的股票名称：MCD
Consumer Staples 加入时间最早的股票名称：GIS
Energy 加入时间最早的股票名称：SLB
Financials 加入时间最早的股票名称：JPM
Health Care 加入时间最早的股票名称：ABT
Industrials 加入时间最早的股票名称：HON
Information Technology 加入时间最早的股票名称：HPQ
Materials 加入时间最早的股票名称：SHW
Real Estate 加入时间最早的股票名称：AVB
Telecommunications Services 加入时间最早的股票名称：T
Utilities 加入时间最早的股票名称：DUK


# 3. merge!

现在你需要同时处理来自两个表中的信息了

1. 请思考，合并两个表的信息的时候，我们应该用什么样的准则对其它们（10分）
2. 请列举每个sector在2013-2016年累计Research&Development的总投入（10分）
3. 请列举出每个sector中，在2013-2016年累计Research&development投入最大的3家公司的名称以及投入的数值（20分）

1、若表格之间有共同的列，则对共同的列进行合并
2、若表格之间没有共同的列，则尽量取表格的并集

In [7]:
fun_df.columns = fun_df.columns.str.lower() # 将所有列名小写化
sec_df.columns = sec_df.columns.str.lower() # 将所有列名小写化
new_df = pd.merge(fun_df, sec_df, on = 'ticker symbol', how = 'inner') # 按股票代码（ticker symbol）进行合并

In [8]:
new_df.rename(columns = {'for year': 'for_year'}, inplace=True)
_1316_data = new_df.query('for_year > 2013 & for_year < 2016') # 筛选出2013-2016年的数据
for key, value in _1316_data.groupby('gics sector'): # 按'GICS Sector'分组并迭代
    print('{} 在2013-2016年累计Research&Development的总投入：{}'.format(key, value['research and development'].sum()))

Consumer Discretionary 在2013-2016年累计Research&Development的总投入：1287664000.0
Consumer Staples 在2013-2016年累计Research&Development的总投入：484500000.0
Energy 在2013-2016年累计Research&Development的总投入：0.0
Financials 在2013-2016年累计Research&Development的总投入：0.0
Health Care 在2013-2016年累计Research&Development的总投入：42390489000.0
Industrials 在2013-2016年累计Research&Development的总投入：2836300000.0
Information Technology 在2013-2016年累计Research&Development的总投入：108638976000.0
Materials 在2013-2016年累计Research&Development的总投入：6052822000.0
Real Estate 在2013-2016年累计Research&Development的总投入：0.0
Telecommunications Services 在2013-2016年累计Research&Development的总投入：0.0
Utilities 在2013-2016年累计Research&Development的总投入：0.0


# 4. 现在让我们来看看更加复杂的数据

请导入price.csv，然后结合你的聪明才智回答以下问题（附加题，40分）

假设你是某基金公司的老板，现在对于每只股票，你都专门安排了一位负责它的交易员。公司规定每一位交易员手中的资金要么全部买入要么全部卖出（空仓，转化为现金）。假设2016年每一位交易员手中都有10000美元，假设他们都能够看到2016年全年的数据，假设他们都能抓住每一次机会，那么请问2016年底时，赚钱最多的股票是哪一只，赚了多少钱？

In [9]:
price_df = pd.read_csv('nyse/prices.csv')

In [10]:
def earn_money(price_seq):
    L = len(price_seq)
    money = 10000
    for i in range(L-1):
        p, next_p = price_seq[i], price_seq[i+1]
        if next_p > p:
            money *= next_p / p
    return money

def invest_symbol(tseq):
    # tseq = price_df[price_df['symbol'] == s][['date', 'open']]
    tseq = tseq[tseq['date'] >= '2016-01-01']
    tseq = tseq.set_index('date')
    # tseq.index = tseq['date']
    # tseq.drop('date', axis = 1, inplace = True)
    tseq = tseq.values.ravel().tolist()
    money = earn_money(tseq)
    return money

In [11]:
group_price = price_df.groupby('symbol')[['date', 'open']].apply(invest_symbol)
print('2016年底时，赚钱最多的股票是{}，赚了{}'.format(group_price.idxmax(), group_price.max()))

2016年底时，赚钱最多的股票是CHK，赚了8616435.240507381
